<a href="https://colab.research.google.com/github/poojagolande256-ui/LLM_RAG_CHATBOT/blob/main/CHATBOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio langchain langchain-google-genai langchain-cohere chromadb pypdf cohere

Build Rag Pipeline App

In [3]:
import gradio as gr
from langchain_google_genai import GoogleGenerativeAI
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
import tempfile
import os
from google.colab import userdata

In [4]:
#Initialize gemini and cohere embedding
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ['COHERE_API_KEY'] = userdata.get('COHERE_API_KEY')
llm = GoogleGenerativeAI(model='gemini-2.5-flash', temperature=0.7)
embeddings = CohereEmbeddings(model='embed-english-v3-0')

In [5]:
from langchain_community.document_loaders.onedrive_file import CHUNK_SIZE
#Initialize qa chain
qa_chain=None

#Process the pdf->load the pdf,divide the pdf into chunks,save in vector database
#Load the pdf
def process_pdf(pdf_file):
    global qa_chain
    temp_path=pdf_file.name if hasattr(pdf_file,'name') else pdf_file
    loader=PyPDFLoader(temp_path)
    documents=loader.load()
#Convert documents into chunks
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks=text_splitter.split_documents(documents)
#save the chunks in vector database
vectorstore=Chroma.from_documents(chunks,embeddings)
#RAG Pipeline with qa chain
qa_chain=RetrievalQA.(llm=llm,chain_type='stuff',retriever=vectorstore.as_retriever(search_kwarge={'k':4}),
                      return_source_documents=True)
return f"Processed {len(chunks)} chunks from pdf"

def query_rag(question):
  if qa_chian is None:
    return "No pdf loaded"

  result =qa_chain.invoke({"query":question})
  answer=result["result"]
  sources ="\n\n----\nSources:\n"
  for i,doc in enumerate(result["source_documents"],1):
    sources+=f"{i}.page {doc.metadata.get('page','N/A')}: {doc.page_content[:100]}...\n"
  return answer+sources

NameError: name 'documents' is not defined